#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

#### 2. Import users table:

In [2]:
users = pd.read_csv('users_table.csv')
users.shape

(40325, 14)

#### 3. Rename Id column to userId

In [3]:
users.rename(columns={'Id': 'userId'}, inplace= True)
users.dtypes

userId               int64
Reputation           int64
CreationDate        object
DisplayName         object
LastAccessDate      object
WebsiteUrl          object
Location            object
AboutMe             object
Views                int64
UpVotes              int64
DownVotes            int64
AccountId            int64
Age                float64
ProfileImageUrl     object
dtype: object

#### 4. Import posts table:

In [4]:
posts = pd.read_csv('posts_table.csv')
posts.dtypes

Id                         int64
PostTypeId                 int64
AcceptedAnswerId         float64
CreaionDate               object
Score                      int64
ViewCount                float64
Body                      object
OwnerUserId              float64
LasActivityDate           object
Title                     object
Tags                      object
AnswerCount              float64
CommentCount               int64
FavoriteCount            float64
LastEditorUserId         float64
LastEditDate              object
CommunityOwnedDate        object
ParentId                 float64
ClosedDate                object
OwnerDisplayName          object
LastEditorDisplayName     object
dtype: object

#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts.rename(columns= {'Id': 'postId', 'OwnerUserId': 'userId'}, inplace= True)

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:
users = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users_posts = users.merge(posts, on='userId')
users_posts.shape

(38962, 9)

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [8]:
print(users_posts.isnull().sum())
print(posts.columns)
# The values are missed in the ViewCount column originating from the posts dataframe.

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64
Index(['postId', 'Score', 'userId', 'ViewCount', 'CommentCount'], dtype='object')


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [9]:
users_posts['ViewCount'].fillna(0, inplace=True)
print(users_posts['ViewCount'].value_counts())

0.0        23572
98.0          43
150.0         43
122.0         42
108.0         41
           ...  
1664.0         1
1337.0         1
64481.0        1
2313.0         1
3357.0         1
Name: ViewCount, Length: 3403, dtype: int64


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [10]:
users_posts = users_posts.astype({'ViewCount': 'int64'})
users_posts.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object